In [1]:
from huggingface_hub import hf_hub_download
hf_hub_download(repo_id="BlinkDL/rwkv-4-pile-14b", filename="RWKV-4-Pile-14B-20230213-8019.pth", cache_dir="/home/kyle/HF-MODEL/rwkv-4-pile-14b")

/home/kyle/miniconda3/envs/kyle-elk-rwkv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'/home/kyle/HF-MODEL/rwkv-4-pile-14b/models--BlinkDL--rwkv-4-pile-14b/snapshots/939b6851f96122b7b49bd00d446b3b49481214dd/RWKV-4-Pile-14B-20230213-8019.pth'

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "5"
os.environ["RWKV_JIT_ON"] = '1'
os.environ["RWKV_CUDA_ON"] = '0' # if '1' then use CUDA kernel for seq mode (much faster)
from rwkv.model import RWKV                         # pip install rwkv
model = RWKV(model='/home/kyle/HF-MODEL/rwkv-4-pile-1b5/models--BlinkDL--rwkv-4-pile-1b5/snapshots/6ea995eaa87a17af560c9b41ce1a3d92355c5a49/RWKV-4-Pile-1B5-20220903-8040.pth', strategy='cuda fp16')

out, state = model.forward([187, 510, 1563, 310, 247], None)   # use 20B_tokenizer.json
print(out.detach().cpu().numpy())                   # get logits
out, state = model.forward([187, 510], None)
out, state = model.forward([1563], state)           # RNN has state (use deepcopy if you want to clone it)
out, state = model.forward([310, 247], state)
print(out.detach().cpu().numpy())                   # same result as above

RWKV_JIT_ON 1 RWKV_CUDA_ON 0 RESCALE_LAYER 6

Loading /home/kyle/HF-MODEL/rwkv-4-pile-1b5/models--BlinkDL--rwkv-4-pile-1b5/snapshots/6ea995eaa87a17af560c9b41ce1a3d92355c5a49/RWKV-4-Pile-1B5-20220903-8040.pth ...
Strategy: (total 24+1=25 layers)
* cuda [float16, float16], store 25 layers
0-cuda-float16-float16 1-cuda-float16-float16 2-cuda-float16-float16 3-cuda-float16-float16 4-cuda-float16-float16 5-cuda-float16-float16 6-cuda-float16-float16 7-cuda-float16-float16 8-cuda-float16-float16 9-cuda-float16-float16 10-cuda-float16-float16 11-cuda-float16-float16 12-cuda-float16-float16 13-cuda-float16-float16 14-cuda-float16-float16 15-cuda-float16-float16 16-cuda-float16-float16 17-cuda-float16-float16 18-cuda-float16-float16 19-cuda-float16-float16 20-cuda-float16-float16 21-cuda-float16-float16 22-cuda-float16-float16 23-cuda-float16-float16 24-cuda-float16-float16 
emb.weight                        f16      cpu  50277  2048 
blocks.0.ln1.weight               f16   cuda:0   2048       

In [3]:
from transformers import AutoTokenizer, GPT2TokenizerFast
tokenizer = GPT2TokenizerFast(tokenizer_file='20B_tokenizer.json')
text = "The following movie review expresses what sentiment? George P. Cosmatos\' \"Rambo: First Blood Part II\" is pure wish-fulfillment. The United States clearly didn\'t win the war in Vietnam. They caused damage to this country beyond the imaginable and this movie continues the fairy story of the oh-so innocent soldiers. The only bad guys were the leaders of the nation, who made this war happen. The character of Rambo is perfect to notice this. He is extremely patriotic, bemoans that US-Americans didn\'t appreciate and celebrate the achievements of the single soldier, but has nothing but distrust for leading officers and politicians. Like every film that defends the war (e.g. \"We Were Soldiers\") also this one avoids the need to give a comprehensible reason for the engagement in South Asia. And for that matter also the reason for every single US-American soldier that was there. Instead, Rambo gets to take revenge for the wounds of a whole nation. It would have been better to work on how to deal with the memories, rather than suppressing them. \"Do we get to win this time?\" Yes, you do.\n\nnegative"

In [4]:
# Return as tensor
tokens = tokenizer(text, return_tensors="pt")
out, state = model.forward(tokens["input_ids"], None)
state[-1].shape # torch.Size([241, 2048]) w/ tensor

torch.Size([241, 2048])

In [5]:
print(f"Received a list of {len(state)} tensors, each of shape {state[0].shape}")

Received a list of 120 tensors, each of shape torch.Size([241, 2048])


In [6]:
# Return as list
tokens = tokenizer(text)
out, state = model.forward(tokens["input_ids"], None)
state[-1].shape # torch.Size([2048]) w/o tensor

torch.Size([2048])

In [7]:
print(f"Received a list of {len(state)} tensors, each of shape {state[0].shape}")

Received a list of 120 tensors, each of shape torch.Size([2048])


In [8]:
len(state)

120

In [9]:
tokenizer.decode(out.argmax())

'\n'

In [10]:
state[-1]

tensor([ 0.3020, -0.3469, -0.0307,  ...,  0.1010,  0.0585, -0.1140],
       device='cuda:0', dtype=torch.float16)

In [11]:
state[-1].shape

torch.Size([2048])

In [12]:
len(state)

120

In [13]:
out, state = model.forward([187, 510, 1563, 310, 247], None)   # use 20B_tokenizer.json
print(out.detach().cpu().numpy())                   # get logits
out, state = model.forward([187, 510], None)
out, state = model.forward([1563], state)           # RNN has state (use deepcopy if you want to clone it)
out, state = model.forward([310, 247], state)

[ -8.828125  -21.828125   -9.359375  ...  -7.7070312  -4.875
  -1.9785156]


In [14]:
from rwkv.utils import PIPELINE, PIPELINE_ARGS
pipeline = PIPELINE(model, "20B_tokenizer.json") 
ctx = "\nIn a shocking finding, scientist discovered a herd of dragons living in a remote, previously unexplored valley, in Tibet. Even more surprising to the researchers was the fact that the dragons spoke perfect Chinese."
print(ctx, end='')

def my_print(s):
    print(s, end='', flush=True)

args = PIPELINE_ARGS(temperature = 1.0, top_p = 0.7, top_k = 100, # top_k = 0 then ignore
                     alpha_frequency = 0.25,
                     alpha_presence = 0.25,
                     token_ban = [0], # ban the generation of some tokens
                     token_stop = [], # stop generation whenever you see any token here
                     chunk_len = 256) # split input into chunks to save VRAM (shorter -> slower)

pipeline.generate(ctx, token_count=200, args=args, callback=my_print)


In a shocking finding, scientist discovered a herd of dragons living in a remote, previously unexplored valley, in Tibet. Even more surprising to the researchers was the fact that the dragons spoke perfect Chinese.

Scientists discovered a herd of dragons living in a remote, previously unexplored valley, in Tibet. Even more surprising to the researchers was the fact that the dragons spoke perfect Chinese.

This discovery is an amazing example of the effects of rapid evolution and natural selection. It is also interesting because it confirms that life can adapt to different environments.

The population of these large dragons ranged from about 200 to about 600 in the area where they live. The most striking aspect of this dragon is its ability to rapidly change its vocalizations from "coo" to "thud." The Tibetan dragon also has long claws that help it hunt on land and on water. They also are able to reach speeds of up to 50 kilometers per hour. The long and pointed snout allows them to 

'\n\nScientists discovered a herd of dragons living in a remote, previously unexplored valley, in Tibet. Even more surprising to the researchers was the fact that the dragons spoke perfect Chinese.\n\nThis discovery is an amazing example of the effects of rapid evolution and natural selection. It is also interesting because it confirms that life can adapt to different environments.\n\nThe population of these large dragons ranged from about 200 to about 600 in the area where they live. The most striking aspect of this dragon is its ability to rapidly change its vocalizations from "coo" to "thud." The Tibetan dragon also has long claws that help it hunt on land and on water. They also are able to reach speeds of up to 50 kilometers per hour. The long and pointed snout allows them to grasp and consume their prey by crushing it between their teeth and then swallowing the flesh. Their tongue is able to carry food items that they swallow whole, such as fruit, seeds, insects, worms,'

In [15]:
pipeline.encode(ctx) == GPT2TokenizerFast(tokenizer_file='20B_tokenizer.json').encode(ctx)

True

In [16]:
import os
from rwkv_hf import RWKVModel
from transformers import GPT2TokenizerFast

os.environ["CUDA_VISIBLE_DEVICES"] = "4"

tokenizer = GPT2TokenizerFast(tokenizer_file='20B_tokenizer.json')
bad_review_text = 'George P. Cosmatos\' "Rambo: First Blood Part II" is pure wish-fulfillment. The United States clearly didn\'t win the war in Vietnam. They caused damage to this country beyond the imaginable and this movie continues the fairy story of the oh-so innocent soldiers. The only bad guys were the leaders of the nation, who made this war happen. The character of Rambo is perfect to notice this. He is extremely patriotic, bemoans that US-Americans didn\'t appreciate and celebrate the achievements of the single soldier, but has nothing but distrust for leading officers and politicians. Like every film that defends the war (e.g. "We Were Soldiers") also this one avoids the need to give a comprehensible reason for the engagement in South Asia. And for that matter also the reason for every single US-American soldier that was there. Instead, Rambo gets to take revenge for the wounds of a whole nation. It would have been better to work on how to deal with the memories, rather than suppressing them. "Do we get to win this time?" Yes, you do.\n\nDid the reviewer find this movie good or bad? bad'
good_review_text = 'George P. Cosmatos\' "Rambo: First Blood Part II" is pure wish-fulfillment. The United States clearly didn\'t win the war in Vietnam. They caused damage to this country beyond the imaginable and this movie continues the fairy story of the oh-so innocent soldiers. The only bad guys were the leaders of the nation, who made this war happen. The character of Rambo is perfect to notice this. He is extremely patriotic, bemoans that US-Americans didn\'t appreciate and celebrate the achievements of the single soldier, but has nothing but distrust for leading officers and politicians. Like every film that defends the war (e.g. "We Were Soldiers") also this one avoids the need to give a comprehensible reason for the engagement in South Asia. And for that matter also the reason for every single US-American soldier that was there. Instead, Rambo gets to take revenge for the wounds of a whole nation. It would have been better to work on how to deal with the memories, rather than suppressing them. "Do we get to win this time?" Yes, you do.\n\nDid the reviewer find this movie good or bad? good'
model = RWKVModel()

RWKV_JIT_ON 1 RWKV_CUDA_ON 0 RESCALE_LAYER 6

Loading /home/kyle/HF-MODEL/rwkv-4-pile-1b5/models--BlinkDL--rwkv-4-pile-1b5/snapshots/6ea995eaa87a17af560c9b41ce1a3d92355c5a49/RWKV-4-Pile-1B5-20220903-8040.pth ...
Strategy: (total 24+1=25 layers)
* cuda [float16, float16], store 25 layers
0-cuda-float16-float16 1-cuda-float16-float16 2-cuda-float16-float16 3-cuda-float16-float16 4-cuda-float16-float16 5-cuda-float16-float16 6-cuda-float16-float16 7-cuda-float16-float16 8-cuda-float16-float16 9-cuda-float16-float16 10-cuda-float16-float16 11-cuda-float16-float16 12-cuda-float16-float16 13-cuda-float16-float16 14-cuda-float16-float16 15-cuda-float16-float16 16-cuda-float16-float16 17-cuda-float16-float16 18-cuda-float16-float16 19-cuda-float16-float16 20-cuda-float16-float16 21-cuda-float16-float16 22-cuda-float16-float16 23-cuda-float16-float16 24-cuda-float16-float16 
emb.weight                        f16      cpu  50277  2048 
blocks.0.ln1.weight               f16   cuda:0   2048       

In [22]:
tokens = tokenizer(bad_review_text, return_tensors="pt")
print(f"Input IDs shape = {tokens}")

states = model(tokens["input_ids"])
tensor_final_state = states[-1]
len(tensor_final_state)

Input IDs shape = {'input_ids': tensor([[23108,   367,    15, 18084,  2056,   375,     8,   346,    51, 47311,
            27,  3973, 14169,  3512,  3719,     3,   310,  6313,  5730,    14,
          1020,  9337,   420,    15,   380,  1986,  2077,  4518,  1904,   626,
          3330,   253,  2137,   275, 15732,    15,  1583,  4269,  4723,   281,
           436,  2586,  4457,   253, 30087,   494,   285,   436,  6440,  7788,
           253, 33784,  2926,   273,   253, 12506,    14,   601, 15377,  9647,
            15,   380,   760,  3076,  6068,   497,   253,  7038,   273,   253,
          5674,    13,   665,  1160,   436,  2137,  5108,    15,   380,  1894,
           273,   416, 47311,   310,  3962,   281,  4366,   436,    15,   754,
           310,  6685, 50174,    13,   320,  6972,   507,   326,  1982,    14,
         34436,  1904,   626, 11435,   285, 17019,   253, 26751,   273,   253,
          2014, 15796,    13,   533,   556,  2717,   533, 48655,   323,  4283,
          6251,   28

121

In [35]:
tokens = tokenizer(bad_review_text, return_tensors="pt")
print(f"Input IDs shape = {tokens['input_ids'].shape}")
# tokens

states = model(tokens["input_ids"])
tensor_final_state = states[-1]
print(f"Received a list of {len(state)} tensors, each of shape {state[0].shape}")

Input IDs shape = torch.Size([1, 243])
Received a list of 120 tensors, each of shape torch.Size([2048])


In [32]:
state[0][0]

tensor(-0.0183, device='cuda:0', dtype=torch.float16)